In [1]:
import pandas as pd
from psifr import fr
import numpy as np

cdcatbeh = pd.read_csv('../../data/cdcatbeh_psifr.csv')

# make sure position column is coded as an integer
cdcatbeh['position'] = cdcatbeh['position'].values.astype(int)

# reindex recall events after excluding intrusions
intrusion_events = cdcatbeh.loc[(cdcatbeh['intrusion'] != '0') & (cdcatbeh['trial_type'] == 'recall')]
intrusion_subjects = intrusion_events['subject'].values
intrusion_lists = intrusion_events['list'].values
intrusion_positions = intrusion_events['position'].values

for i in range(len(intrusion_subjects)):
    cdcatbeh.loc[(cdcatbeh.subject==intrusion_subjects[i]) & (cdcatbeh.list==intrusion_lists[i]) & (cdcatbeh.position > intrusion_positions[i]) & (cdcatbeh.trial_type == 'recall'), 'position'] -= 1
cdcatbeh = cdcatbeh.loc[(cdcatbeh['intrusion'] == '0') | (cdcatbeh['trial_type'] == 'study')]

# move item string representation to a separate column
cdcatbeh['item_string'] = cdcatbeh['item']
cdcatbeh['item_string_index'] = fr.pool_index(cdcatbeh['item_string'], pd.unique(cdcatbeh['item_string'])).values.astype(int)

# reindex items based on initual study position
for each in cdcatbeh.groupby(['subject', 'list']):
    cdcatbeh.loc[(cdcatbeh.subject==each[0][0]) &( cdcatbeh.list==each[0][1]), 'item'] = fr.pool_index(
    each[1]['item'], pd.unique(each[1]['item']))


merged = fr.merge_free_recall(
    cdcatbeh, 
    list_keys=['category', 'list_type', 'item_string', 'item_string_index']
    )
merged = merged.drop(columns=['prior_list', 'prior_input'])
#merged = merged.dropna(axis='index', subset='input')
merged.to_csv('../../data/CDCATBEH.csv', index=False)

# # re index so all trials ascribed to one subject
# subject_index = 1
# trial_index = 0
# for each in cdcatbeh.groupby(['subject', 'list']):
#     trial_index += 1
#     cdcatbeh.loc[(cdcatbeh.subject==each[0][0]) & (cdcatbeh.list==each[0][1]), 'list'] = trial_index
# cdcatbeh['subject'] = subject_index

# agg_merged = fr.merge_free_recall(cdcatbeh, list_keys=['category', 'list_type', 'item_string', 'item_string_index'])
# agg_merged = agg_merged.drop(columns=['prior_list', 'prior_input'])
# agg_merged.to_csv('../../data/AggCDCATBEH.csv', index=False)

c:\ProgramData\Miniconda3\lib\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)


In [2]:
cdcatbeh

,subject,list,item,list_type,category,intrusion,position,trial_type,item_string,item_string_index
0,1,1,0,light,cel,[],1,study,Halle Berry,0
1,1,1,1,light,cel,[],2,study,Simon Cowell,1
2,1,1,2,light,cel,[],3,study,Kristen Stewart,2
3,1,1,3,light,obj,[],4,study,highlighter,3
4,1,1,4,light,obj,[],5,study,acorn,4
...,...,...,...,...,...,...,...,...,...,...
28405,97,8,3,light,obj,0,11,recall,purse,119
28406,97,8,25,light,obj,0,12,recall,staple remover,61
28407,97,8,24,light,obj,0,13,recall,brick,113
28408,97,8,12,light,loc,0,14,recall,Gettysburg,42


In [3]:
cdcatmr_wiki2vec = pd.read_csv('../../data/cdcatmr-wiki2vec.csv')

items = pd.unique(merged['item_string'])
wordpool = np.zeros((len(items)+1, 300))
keys = cdcatmr_wiki2vec['item'].values.tolist()

for item in items:
    i = merged.loc[merged['item_string'] == item, 'item_string_index'].values[0]
    wordpool[i] = cdcatmr_wiki2vec.loc[keys.index(item)].values[2:]
    
np.save('../../data/cdcatmr-wiki2vec.npy', wordpool)

In [4]:
cdcatmr_use = pd.read_csv('../../data/cdcatmr-use.csv')
items = pd.unique(merged['item_string'])
wordpool = np.zeros((len(items)+1, 512))
keys = cdcatmr_use['itemName'].values.tolist()

for item in items:
    i = merged.loc[merged['item_string'] == item, 'item_string_index'].values[0]
    wordpool[i] = cdcatmr_use.loc[keys.index(item)].values[3:]

np.save('../../data/cdcatmr-use.npy', wordpool)

In [5]:
cdcatmr_use = pd.read_csv('../../data/cdcatmr-use.csv')
items = pd.unique(merged['item_string'])
wordpool = np.zeros((len(items)+1, 3))
keys = cdcatmr_use['itemName'].values.tolist()

for item in items:
    i = merged.loc[merged['item_string'] == item, 'item_string_index'].values[0]
    category_index = cdcatmr_use.loc[keys.index(item)].values[2]-1
    wordpool[i, category_index] = 1

np.save('../../data/cdcatmr-cat.npy', wordpool)